In [1]:
#TODO
# Try meshing some other polydata
# See what is going on with caps?

#read https://github.com/neilbalch/SimVascular-pythondemos/tree/master/Testing%20Framework
import os
import sys

sys.path.append(os.path.abspath('..'))

import modules.io as io

CUR_DIR = os.getcwd()
print(CUR_DIR)
SV_PATH       = '/home/marsdenlab/projects/SV/fork/SimVascular/Python/site-packages'
SV_BUILD_PATH = '/home/marsdenlab/projects/SV/fork/build/SimVascular-build/lib'

sys.path.append(SV_PATH)

#chdir needed to find sv shared object files (.so)
os.chdir(SV_BUILD_PATH)

import sv
#import modules.mesh_util as mesh_util

#os.chdir(CUR_DIR)
# print("\n sv")
# print(dir(sv))

# print("\n sv Geom")
# print(dir(sv.Geom))

# print("\n sv Solid")
# print(dir(sv.Solid))

# print("\n sv SolidPolyData")
# print(dir(sv.SolidPolyData))

# print("\n sv MeshTetgen")
# print(dir(sv.MeshTetgen))

# print("\n sv MeshUtil")
# print(dir(sv.MeshUtil))

/home/marsdenlab/projects/seg_regression/notebooks
org_sv_pythondatanodes was not found!


In [2]:
GROUPS_FILES = [
 '/home/marsdenlab/projects/SV/UQ/data/2/aorta_corrected.json',
 '/home/marsdenlab/projects/SV/UQ/data/2/right_iliac_corrected.json'    
]

GROUPS = [io.load_json(f) for f in GROUPS_FILES]

PATH_FILES  = [
 '/home/marsdenlab/projects/SV/UQ/data/paths/aorta.txt',
 '/home/marsdenlab/projects/SV/UQ/data/paths/right_iliac.txt'
]

PATHS = [io.parsePathPointsFile(f) for f in PATH_FILES]

NAMES = [
    "aorta",
    "right_iliac"
]

INTERVAL = 10

In [3]:
len(PATHS[0])

30

In [4]:
#for path in paths, create path, add points
#for group in groups, create group, add points
pd_names = {}
for name,path,group in zip(NAMES, PATHS, GROUPS):
    print(name)
    
    #path
    sv_path = sv.Path.pyPath()
    sv_path.NewObject(name)
    
    for point in path:
        p = list(point)
        sv_path.AddPoint(p[1:4])
        
    sv_path.CreatePath()
    print(sv_path.GetPathPtsNum())
    
    #contour
    kernel = "SplinePolygon"
    sv.Contour.SetContourKernel(kernel)
    
    path_pts = sv_path.GetPathPosPts()

    point_per_id = (sv_path.GetPathPtsNum()-1)/(len(path)-1)
    point_per_id = int(point_per_id)
    
    print(point_per_id)
    
    pd_names[name] = []
    
    for i,t in enumerate(group.items()):
        k,v = t
        k = int(k)
        group_pos = int(k/INTERVAL)
        pos = int(group_pos*point_per_id)

        contour = sv.Contour.pyContour()

        contour_name = name+'_'+str(pos)
        
        contour.NewObject(contour_name, name, pos)
        
        ctrlPts = v
        contour.SetCtrlPts(v)
        contour.Create()
        
        pd_name = contour_name+'_pd'
        pd_names[name].append(pd_name)
        contour.GetPolyData(pd_name)

aorta
117
4
right_iliac
115
6


In [5]:
print(pd_names)
#sorted(sv.Repository.List())

{'aorta': ['aorta_0_pd', 'aorta_4_pd', 'aorta_8_pd', 'aorta_12_pd', 'aorta_16_pd', 'aorta_20_pd', 'aorta_24_pd', 'aorta_28_pd', 'aorta_32_pd', 'aorta_36_pd', 'aorta_40_pd', 'aorta_44_pd', 'aorta_48_pd', 'aorta_52_pd', 'aorta_60_pd', 'aorta_64_pd', 'aorta_68_pd', 'aorta_72_pd', 'aorta_76_pd', 'aorta_80_pd', 'aorta_84_pd', 'aorta_88_pd', 'aorta_92_pd', 'aorta_96_pd', 'aorta_100_pd', 'aorta_104_pd', 'aorta_108_pd', 'aorta_112_pd', 'aorta_116_pd'], 'right_iliac': ['right_iliac_0_pd', 'right_iliac_18_pd', 'right_iliac_24_pd', 'right_iliac_30_pd', 'right_iliac_36_pd', 'right_iliac_42_pd', 'right_iliac_48_pd', 'right_iliac_54_pd', 'right_iliac_60_pd', 'right_iliac_66_pd', 'right_iliac_72_pd', 'right_iliac_78_pd', 'right_iliac_84_pd', 'right_iliac_90_pd', 'right_iliac_96_pd', 'right_iliac_102_pd', 'right_iliac_108_pd']}


In [6]:
#lofting
numOutPtsInSegs = 60
numOutPtsAlongLength = 100
numPtsInLinearSampleAlongLength = 240
numLinearPtsAlongLength = 120
numModes = 20
useFFT = 0
useLinearSampleAlongLength = 1

sample_names = {}
aligned_names = {}
for name in NAMES:
    sample_names[name] = []
    aligned_names[name] = []
    
    for pd_name in pd_names[name]:
        s_name = pd_name+'_s'
        sv.Geom.SampleLoop(pd_name,numOutPtsInSegs,s_name)
        sample_names[name].append(s_name)

    #align contours
    n1 = sample_names[name][0]
    aligned_names[name].append(n1)
    for i in range(1,len(sample_names[name])):
        n2 = sample_names[name][i]
        na = n2+'_aligned'
        sv.Geom.AlignProfile(n1,n2,na,0)
        aligned_names[name].append(na)
        
        n1 = na
        
    #loft contours
    srcList = aligned_names[name]
    dstName = name+'_loft'

    sv.Geom.LoftSolid(srcList,dstName,numOutPtsInSegs,
                   numOutPtsAlongLength,numLinearPtsAlongLength,
                   numModes,useFFT,useLinearSampleAlongLength)

In [7]:
print(aligned_names)

{'aorta': ['aorta_0_pd_s', 'aorta_4_pd_s_aligned', 'aorta_8_pd_s_aligned', 'aorta_12_pd_s_aligned', 'aorta_16_pd_s_aligned', 'aorta_20_pd_s_aligned', 'aorta_24_pd_s_aligned', 'aorta_28_pd_s_aligned', 'aorta_32_pd_s_aligned', 'aorta_36_pd_s_aligned', 'aorta_40_pd_s_aligned', 'aorta_44_pd_s_aligned', 'aorta_48_pd_s_aligned', 'aorta_52_pd_s_aligned', 'aorta_60_pd_s_aligned', 'aorta_64_pd_s_aligned', 'aorta_68_pd_s_aligned', 'aorta_72_pd_s_aligned', 'aorta_76_pd_s_aligned', 'aorta_80_pd_s_aligned', 'aorta_84_pd_s_aligned', 'aorta_88_pd_s_aligned', 'aorta_92_pd_s_aligned', 'aorta_96_pd_s_aligned', 'aorta_100_pd_s_aligned', 'aorta_104_pd_s_aligned', 'aorta_108_pd_s_aligned', 'aorta_112_pd_s_aligned', 'aorta_116_pd_s_aligned'], 'right_iliac': ['right_iliac_0_pd_s', 'right_iliac_18_pd_s_aligned', 'right_iliac_24_pd_s_aligned', 'right_iliac_30_pd_s_aligned', 'right_iliac_36_pd_s_aligned', 'right_iliac_42_pd_s_aligned', 'right_iliac_48_pd_s_aligned', 'right_iliac_54_pd_s_aligned', 'right_iliac_6

In [8]:
#Set solid kernel
sv.VMTKUtils.Cap_with_ids('aorta_loft','aorta_loft_cap',0,0)
sv.VMTKUtils.Cap_with_ids('right_iliac_loft','right_iliac_loft_cap',0,0)

sv.Geom.Intersect("aorta_loft_cap", "right_iliac_loft_cap", "model_loft_cap")

sv.Geom.Set_array_for_local_op_sphere('model_loft_cap','model_loft_cap_1',3,
                                      [1.27,3.37,-8.65])
sv.Geom.Local_constrain_smooth('model_loft_cap_1',
                               'model_loft_cap_2',5,0.8)
    
sv.Solid.SetKernel('PolyData')
#Create model from polydata
solid = sv.Solid.pySolidModel()
solid.NewObject('model_pd')
solid.SetVtkPolyData('model_loft_cap_2')

#Extract boundary faces
solid.GetBoundaryFaces(45)
print ("Creating model: \nFaceID found: " + str(solid.GetFaceIds()))
#Write to file 
solid.WriteNative("/home/marsdenlab/projects/SV/UQ/data/pymodel.exterior.vtp")

Creating model: 
FaceID found: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']


True

In [9]:
print(solid.GetFaceIds())

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']


In [10]:
#Set mesh kernel
sv.MeshObject.SetKernel('TetGen')

#Create mesh object
msh = sv.MeshObject.pyMeshObject()
msh.NewObject('mesh')

#Load Model
solidFn = "/home/marsdenlab/projects/SV/UQ/data/pymodel.exterior.vtp"
msh.LoadModel(solidFn)

#Create new mesha
msh.NewMesh()
msh.SetMeshOptions('SurfaceMeshFlag',[1])
msh.SetMeshOptions('VolumeMeshFlag',[1])
msh.SetMeshOptions('GlobalEdgeSize',[0.25])
msh.SetMeshOptions('MeshWallFirst',[1])
msh.GenerateMesh()

# #Save mesh to file
ug_fn = "/home/marsdenlab/projects/SV/UQ/data/mesh_ug.vtk"
pd_fn = "/home/marsdenlab/projects/SV/UQ/data/mesh_pd.vtk"

mesh_ug_name = 'mesh_ug'
mesh_pd_name = 'mesh_pd'

msh.GetUnstructuredGrid(mesh_ug_name)
msh.GetPolyData(mesh_pd_name)

sv.Repository.WriteVtkUnstructuredGrid(mesh_ug_name,"ascii",ug_fn)
sv.Repository.WriteVtkPolyData(mesh_pd_name,"ascii",pd_fn)

SystemError: <built-in method GenerateMesh of pyMeshObject.pyMeshObject object at 0x7fbe8b096378> returned a result with an error set